<a href="https://colab.research.google.com/github/emkafie/Machine-Learning/blob/main/TP_JS11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tugas**

## **Klasifikasi Data Voice**

In [1]:
import cv2
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from pathlib import Path
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
# 1. Load Data
try:
    df = pd.read_csv('voice.csv')
except FileNotFoundError:
    print("File voice.csv tidak ditemukan. Pastikan file berada di direktori yang sama.")
    exit()

# 2. Preprocessing
# Encoding label (Male/Female -> 1/0)
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

X = df.drop('label', axis=1)
y = df['label']

# 3. Definisi Skenario
ratios = [0.3, 0.2] # 0.3 artinya 70:30, 0.2 artinya 80:20
kernels = ['linear', 'poly', 'rbf']
results = []

# 4. Loop Training dan Evaluasi
for test_size in ratios:
    # Split Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Scaling (Penting untuk SVM)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    split_name = f"{int((1-test_size)*100)}:{int(test_size*100)}"

    for kernel in kernels:
        # Inisiasi dan Training Model
        svm = SVC(kernel=kernel)
        svm.fit(X_train, y_train)

        # Prediksi dan Akurasi
        y_pred = svm.predict(X_test)
        acc = accuracy_score(y_test, y_pred)

        # Simpan Hasil
        results.append({
            'Split Rasio': split_name,
            'Kernel': kernel,
            'Akurasi': acc
        })

# 5. Tabulasi Hasil
results_df = pd.DataFrame(results)
print("--- Hasil Perbandingan Performansi Model SVM ---")
print(results_df)

--- Hasil Perbandingan Performansi Model SVM ---
  Split Rasio  Kernel   Akurasi
0       70:30  linear  0.970557
1       70:30    poly  0.957939
2       70:30     rbf  0.981073
3       80:20  linear  0.976341
4       80:20    poly  0.971609
5       80:20     rbf  0.982650


* Preprocessing: Data label diubah menjadi numerik dan fitur distandarisasi menggunakan StandardScaler. Standarisasi sangat krusial karena SVM sensitif terhadap skala data.

* Looping Skenario: Program melakukan iterasi untuk rasio split 70:30 dan 80:20. Di dalam setiap split, tiga kernel (Linear, Poly, RBF) dilatih.

* Tabel Hasil: Output menampilkan tabel dengan kolom Split Rasio, Kernel, dan Akurasi. Biasanya, kernel RBF atau Linear memberikan hasil yang sangat kompetitif pada dataset ini (di atas 97%), sementara kernel Polynomial mungkin membutuhkan waktu komputasi lebih lama tergantung derajat polinomialnya.

## **Klasifikasi Citra (Siang/Malam) dengan Histogram**

In [5]:
# ==========================================
# 1. FUNGSI EKSTRAKSI FITUR HISTOGRAM
# ==========================================
def extract_histogram_feature(img_dir):
    p = Path(img_dir)
    features = []
    labels = []

    # Mapping label string ke angka
    # day -> 1, night -> 0

    for folder in p.glob('*'):
        label_str = str(folder).split(os.sep)[-1]
        label = 1 if label_str == 'day' else 0

        for file in folder.glob('*.jpg'):
            img = cv2.imread(str(file))
            if img is not None:
                # Resize untuk konsistensi (opsional jika menggunakan histogram, tapi baik untuk performa)
                img = cv2.resize(img, (200, 200))

                # Menghitung Histogram untuk setiap channel (B, G, R)
                # 32 bins per channel cukup untuk menangkap distribusi warna tanpa terlalu banyak fitur
                hist_features = []
                colors = ('b', 'g', 'r')
                for i, col in enumerate(colors):
                    # Hitung histogram: channel i, mask None, 32 bins, range 0-256
                    hist = cv2.calcHist([img], [i], None, [32], [0, 256])
                    # Normalisasi histogram agar tidak terpengaruh ukuran gambar
                    cv2.normalize(hist, hist)
                    hist_features.extend(hist.flatten())

                features.append(hist_features)
                labels.append(label)

    return np.array(features), np.array(labels)

# ==========================================
# 2. LOAD DATA & SPLIT
# ==========================================
# Menggabungkan data train dan test folder untuk displit ulang sesuai instruksi (80:20)
base_dir = "/content/drive/MyDrive/Machine Learning/images/"

# Cek keberadaan folder
if os.path.exists(base_dir):
    # Ekstraksi fitur dari folder training dan test (digabung lalu di-reshuffle)
    X_part1, y_part1 = extract_histogram_feature(base_dir + "training")
    X_part2, y_part2 = extract_histogram_feature(base_dir + "test")

    X = np.concatenate((X_part1, X_part2), axis=0)
    y = np.concatenate((y_part1, y_part2), axis=0)

    print(f"Total Data: {X.shape[0]}, Jumlah Fitur per Gambar: {X.shape[1]}")

    # Split Data 80:20
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scaling Data (Wajib untuk SVM)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # ==========================================
    # 3. HYPERPARAMETER TUNING (GRIDSEARCH)
    # ==========================================
    print("\nMemulai Grid Search...")
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['rbf']
    }

    grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=1, cv=3)
    grid.fit(X_train, y_train)

    # ==========================================
    # 4. EVALUASI
    # ==========================================
    print("\n--- Hasil Terbaik ---")
    print(f"Parameter Terbaik: {grid.best_params_}")

    grid_predictions = grid.predict(X_test)
    accuracy = accuracy_score(y_test, grid_predictions)

    print(f"Akurasi Model (Kernel RBF + Histogram): {accuracy:.4f}")

else:
    print("Folder 'images' tidak ditemukan.")

Total Data: 400, Jumlah Fitur per Gambar: 96

Memulai Grid Search...
Fitting 3 folds for each of 16 candidates, totalling 48 fits

--- Hasil Terbaik ---
Parameter Terbaik: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Akurasi Model (Kernel RBF + Histogram): 0.9750


Ekstraksi Fitur Histogram:
* Berbeda dengan fitur rata-rata kecerahan, histogram menangkap sebaran warna. Fungsi cv2.calcHist digunakan untuk menghitung frekuensi kemunculan intensitas piksel.
* Kode menggunakan 32 bins per kanal warna (R, G, B). Total fitur yang dihasilkan per gambar adalah $32 \times 3 = 96$ fitur. Fitur ini kemudian dinormalisasi agar invarian terhadap ukuran gambar.

Dataset: Data dari folder training dan test digabung terlebih dahulu, kemudian dipecah kembali menggunakan train_test_split dengan rasio 80:20 sesuai instruksi soal.

Grid Search:

* Dilakukan pencarian parameter terbaik untuk C (penalti kesalahan) dan gamma (koefisien kernel RBF).

* Output Parameter Terbaik menampilkan kombinasi hyperparameter yang menghasilkan skor validasi silang tertinggi.

Akurasi Akhir:

* Nilai akurasi pada data testing ditampilkan di akhir. Penggunaan fitur histogram pada SVM kernel RBF umumnya menghasilkan akurasi yang lebih tinggi dibandingkan hanya menggunakan rata-rata kecerahan, karena histogram mampu membedakan karakteristik warna "siang" (biru langit, awan putih) dan "malam" (gelap, lampu kuning) dengan lebih detail.